In [2]:
import folium

In [46]:
kansas_city_coord = [39.085594, -94.585241]  # Kansas City, roughly the geographic center of the USA
san_fransisco_coord = [37.77, -122.41] #[37.77, 122.41]
boston_coord = [42.36, -71.06] #[42.36, 71.06]

# Create a blank map

In [47]:
def create_blank_map():
    m = folium.Map(location=kansas_city_coord, zoom_start=5, tiles='cartodbpositron', width='100%', height='100%')
    return m

In [48]:
create_blank_map()

# Basic markers, circles and lines
Marker with a colored icon, a tooltip and a multi-line popup. 

## Layer control
In order to be able to hide groups of elements, add them to a FeatureGroup.<br>
Then add a LayerControl to the map. This will automatically pickup any FeatureGroups.

## Icons
For ids of icons, see https://fontawesome.com/icons?d=gallery&q=shopp&m=free

## Multi-line popup
Work-around for multi-line in popup: https://github.com/python-visualization/folium/issues/469
`popup = (
        "Time: {time}<br>"
        "Speed: {speed} km/h<br>"
       ).format(time=row.name.strftime('%H:%M'), speed=str(round(row['spd'],2))`

In [63]:
def create_basic_markers_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Basic markers')
    
    # Marker with icon, tooltip and popup
    coord = boston_coord
    icon = folium.Icon(color='green',
        icon_color='white',
        icon='shopping-cart',
        angle=0, prefix='fa')
    popup = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('Boston', 2019)
    tooltip = 'Boston'
    folium.Marker(coord,
        popup=popup,
        tooltip=tooltip,
        icon=icon
        ).add_to(fg)
    
    
    # Circle 1, tooltip and popup
    coord_1 = kansas_city_coord
    popup_1 = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('Kansas', 2019)
    radius_1 = 20
    folium.CircleMarker(coord_1,
        radius=radius_1,
        popup=popup_1,
        tooltip='Kansas',
        color='blue',
        fill_color='green',
        fill=True,
        fill_opacity=0.4,
        stroke=True,
        ).add_to(fg)
    
    # Circle 2, tooltip and popup
    coord_2 = san_fransisco_coord
    popup_2 = (
        "Hello, I am {}.<br>"
        "Welcome to my showcase in {}."
            ).format('San Fransisco', 1972)
    radius_2 = 40
    folium.CircleMarker(coord_2,
        radius=radius_2,
        popup=popup_2,
        tooltip='San Fransisco',
        color='red',
        fill_color='orange',
        fill=True,
        fill_opacity=0.4,
        stroke=True,
        ).add_to(fg)
                 
    # Line
    coordinates = [kansas_city_coord, boston_coord]
    folium.PolyLine(coordinates, popup="Line", tooltip="Line tooltip", color='blue', weight=5).add_to(fg)
    

    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m

In [64]:
create_basic_markers_map()

# Arrow line


In [72]:
def get_bearing(p1, p2):
    """
    Returns compass bearing from p1 to p2

    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon

    Return
    compass bearing of type float

    Notes
    Based on https://gist.github.com/jeromer/2005586
    """
    import numpy as np
    long_diff = np.radians(p2.lon - p1.lon)

    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)

    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2)
         - (np.sin(lat1) * np.cos(lat2)
            * np.cos(long_diff)))

    bearing = np.degrees(np.arctan2(x, y))

    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

def get_arrows(locations, color='blue', size=6, n_arrows=3, add_to=None):
    """Add arrows to a hypothetical line between the first 2 locations in the locations list.
    Get a list of correctly placed and rotated arrows/markers to be plotted.

    Args:
        locations : List of lists of lat lon that represent the
                    start and end of the line.
                    eg [[41.1132, -96.1993],[41.3810, -95.8021]]
                    The locations is a list so that it matches the input for the folium.PolyLine.
        color : Whatever folium can use.  Default is 'blue'
        size : Size of arrow. Default is 6
        n_arrows : Number of arrows to create.  Default is 3.
        add_to: map or FeatureGroup the arrows are added to.

    Returns:
        list of arrows/markers
    """
    # TODO: generalize so that locations can be any length >=2, i.e. a PolyLine with more than 1 section.

    from collections import namedtuple
    import numpy as np
    Point = namedtuple('Point', field_names=['lat', 'lon'])

    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])

    # getting the rotation needed for our marker.
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90

    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows + 1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows + 1]

    arrows = []

    # creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points,
                                                  fill_color=color, number_of_sides=3,
                                                  radius=size, rotation=rotation).add_to(add_to))
    return arrows

In [73]:
def create_arrow_line_map():
    m = create_blank_map()
    fg = folium.FeatureGroup(name='Arrow line')
    
    # Line
    coordinates = [kansas_city_coord, boston_coord]
    folium.PolyLine(coordinates, popup="Line", tooltip="Line tooltip", color='blue', weight=5).add_to(fg)
    
    #Add arrows
    get_arrows(locations=coordinates, color='#FFFFFF', n_arrows=3, add_to=fg)
    
    fg.add_to(m)
    folium.LayerControl(position='topleft').add_to(m)
    return m

In [74]:
create_arrow_line_map()